In [1]:
from __future__ import print_function
import numpy as np
import os
import sys
from six.moves import cPickle as pickle

In [2]:
HUMAN_MOTIF_VARIANTS = [
    'AATAAA',
    'ATTAAA',
    'AAAAAG',
    'AAGAAA',
    'TATAAA',
    'AATACA',
    'AGTAAA',
    'ACTAAA',
    'GATAAA',
    'CATAAA',
    'AATATA',
    'AATAGA'
]

In [3]:
POS_PATH = 'data/human/dragon_polyA_data/positive5fold/'
NEG_PATH = 'data/human/dragon_polyA_data/negatives5fold/'
# POS_PATH = 'human_data/omni_polyA_data/positive/'
# NEG_PATH = 'human_data/omni_polyA_data/negative/'
BATCH_SIZE = 64
PATCH_SIZE = 10
DEPTH = 16
NUM_HIDDEN = 64
SEQ_LEN = 206 + 2*PATCH_SIZE-2
NUM_CHANNELS = 4
NUM_LABELS = 2
NUM_EPOCHS = 200
NUM_FOLDS = 5
HYPER_DICT = None

In [4]:
def get_motif_data(data_root, label):
    data = {}
    labels = {}
    for motif in HUMAN_MOTIF_VARIANTS:
        data[motif] = []
        for data_file in os.listdir(data_root):
            if motif in data_file:
                data_path = os.path.join(data_root, data_file)
                with open(data_path, 'r') as f:
                    alphabet = np.array(['A','G','T','C'])
                    for line in f:
                        line = list(line.strip('\n'))
                        seq = np.array(line, dtype = '|U1').reshape(-1,1)
                        seq_data = (seq == alphabet).astype(np.float32)
                        data[motif].append(seq_data)
        data[motif] = np.stack(data[motif]).reshape([-1,206,1,4])
        if label:
            labels[motif] = np.zeros(data[motif].shape[0])
        else:
            labels[motif] = np.ones(data[motif].shape[0])
    return data, labels

In [5]:
def shuffle(dataset, labels, randomState = None):
    if randomState is None:
        permutation = np.random.permutation(labels.shape[0])
    else:
        permutation = randomState.permutation(labels.shape[0])
    shuffled_data = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_data, shuffled_labels

In [11]:
def produce_motif_dataset(num_folds, pos_path, neg_path, seed=0):
    pos_data, pos_labels = get_motif_data(pos_path, True)
    neg_data, neg_labels = get_motif_data(neg_path, False)
    randomState = np.random.RandomState(seed)
    for motif in HUMAN_MOTIF_VARIANTS:
        pos_data[motif],pos_labels[motif] = shuffle(pos_data[motif], pos_labels[motif], randomState)
        neg_data[motif],neg_labels[motif] = shuffle(neg_data[motif], neg_labels[motif], randomState)
        print('Positive %s:'%motif, pos_data[motif].shape, pos_labels[motif].shape)
        print('Negative %s:'%motif, neg_data[motif].shape, neg_labels[motif].shape)
    return pos_data, pos_labels, neg_data, neg_labels

In [12]:
pos_data, pos_labels, neg_data, neg_labels = produce_motif_dataset(NUM_FOLDS, POS_PATH, NEG_PATH)

Positive AATAAA: (2595, 206, 1, 4) (2595,)
Negative AATAAA: (2595, 206, 1, 4) (2595,)
Positive ATTAAA: (1200, 206, 1, 4) (1200,)
Negative ATTAAA: (1200, 206, 1, 4) (1200,)
Positive AAAAAG: (615, 206, 1, 4) (615,)
Negative AAAAAG: (615, 206, 1, 4) (615,)
Positive AAGAAA: (625, 206, 1, 4) (625,)
Negative AAGAAA: (625, 206, 1, 4) (625,)
Positive TATAAA: (390, 206, 1, 4) (390,)
Negative TATAAA: (390, 206, 1, 4) (390,)
Positive AATACA: (440, 206, 1, 4) (440,)
Negative AATACA: (440, 206, 1, 4) (440,)
Positive AGTAAA: (335, 206, 1, 4) (335,)
Negative AGTAAA: (335, 206, 1, 4) (335,)
Positive ACTAAA: (345, 206, 1, 4) (345,)
Negative ACTAAA: (345, 206, 1, 4) (345,)
Positive GATAAA: (230, 206, 1, 4) (230,)
Negative GATAAA: (230, 206, 1, 4) (230,)
Positive CATAAA: (205, 206, 1, 4) (205,)
Negative CATAAA: (205, 206, 1, 4) (205,)
Positive AATATA: (205, 206, 1, 4) (205,)
Negative AATATA: (205, 206, 1, 4) (205,)
Positive AATAGA: (185, 206, 1, 4) (185,)
Negative AATAGA: (185, 206, 1, 4) (185,)


In [15]:
neg_data

{'AATAAA': array([[[[0., 0., 0., 1.]],
 
         [[1., 0., 0., 0.]],
 
         [[1., 0., 0., 0.]],
 
         ...,
 
         [[1., 0., 0., 0.]],
 
         [[0., 0., 1., 0.]],
 
         [[1., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., 1.]],
 
         [[0., 0., 0., 1.]],
 
         [[0., 0., 1., 0.]],
 
         ...,
 
         [[0., 0., 0., 1.]],
 
         [[1., 0., 0., 0.]],
 
         [[1., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., 1.]],
 
         [[0., 0., 1., 0.]],
 
         [[0., 0., 0., 1.]],
 
         ...,
 
         [[0., 0., 0., 1.]],
 
         [[0., 0., 0., 1.]],
 
         [[0., 0., 1., 0.]]],
 
 
        ...,
 
 
        [[[1., 0., 0., 0.]],
 
         [[1., 0., 0., 0.]],
 
         [[0., 0., 0., 1.]],
 
         ...,
 
         [[1., 0., 0., 0.]],
 
         [[0., 0., 1., 0.]],
 
         [[1., 0., 0., 0.]]],
 
 
        [[[1., 0., 0., 0.]],
 
         [[1., 0., 0., 0.]],
 
         [[0., 1., 0., 0.]],
 
         ...,
 
         [[0., 0., 1., 0.]],
 
         [[0.

In [16]:
neg_labels

{'AATAAA': array([1., 1., 1., ..., 1., 1., 1.]),
 'ATTAAA': array([1., 1., 1., ..., 1., 1., 1.]),
 'AAAAAG': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 